Copyright © 2023 Gurobi Optimization, LLC

# 航空公司航班中断后的重新规划

天气情况是航空业面临的主要威胁。暴风雪、暴雨和结冰跑道的不可预测性使航空规划者难以制定准确的时刻表。
这些事件会导致航班延误和取消,不仅给乘客带来不便,还会给航空公司造成巨大的经济损失。
例如,2014年的极地涡旋据估计给航空业和旅客造成了高达14亿美元的损失[(CNBC, 2014)](https://www.cnbc.com/2014/01/08/weather-flight-disruptions-cost-14-billion-data.html)。
因此,管理天气相关问题并制定应急预案对任何航空公司的成功至关重要。


<!--  <img height="227.4774774774775px" src="https://media3.giphy.com/media/gkAEM5sXCFqB465YWg/giphy.gif?cid=de9bf95e87544fzo1lm01e68i3mxxou95x5zgiewz66hwi45&amp;rid=giphy.gif&amp;ct=g" width="700px" itemtype="http://schema.skype.com/Giphy" key="gif_0"> -->
 
 |<img src="https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/image_snowstorm.jpeg" width="500" align="center">| 
|:--:|
| <b>因天气延误导致的航班取消比我们想象的更常见。图片来源: [Travel Refund](https://travelrefund.com/articles/when-do-flights-get-cancelled-due-to-weather/) </b>| 

假设你是一家航空公司的航班计划人员。对于给定的某一天,你已经为全国各地的航班售出了机票,并且已经制定了机队运营计划来服务所有这些航班。

假设这一天发生了天气事件(如暴风雪),导致机场无法全负荷运转。这意味着一些航班必须取消。当航班取消时,分配给这些航班的飞机需要重新规划路线。所以问题变成了:航空公司如何决定运营/取消哪些航班,以及如何最好地为飞机重新规划路线?

这个问题没有直接的答案,但数学优化可以帮助解决。


本笔记本介绍了在天气中断后决定运营和取消哪些航班的优化问题。
我们通过构建一个**数学优化**模型来减少因取消航班造成的收入损失。
在这个例子中,我们使用了由[Amadeus](https://amadeus.com/en)编译的法国实际数据。

本笔记本分为三个部分。
- 首先,我们读取数据集。
- 其次,我们通过定义**决策变量**、**目标函数**和**约束条件**来构建优化模型。
- 第三,假设某种程度的天气中断,我们求解优化模型以找到新的最优航班计划以及飞机路线。

## 数据


**数据集**: 我们使用由[Amadeus](https://amadeus.com/en)编译的真实数据,该数据是[ROADEF 2009挑战赛: 商业航空中断管理](https://www.roadef.org/challenge/2009/en/)的一部分。此数据集基于法国某航空公司的航班计划。对于本笔记本,我们已对该数据集进行了预处理,并将信息存储为三部分:

- **当前航班计划:** 假设没有天气中断(即所有机场全负荷运转)的情况下,当前计划的航班及其飞机分配。
- **飞机的起始和结束位置:** 每架飞机一天开始时和结束时的位置。这些信息对于确保飞机在第二天需要的位置是必要的,以确保中断不会延续到第二天。
- **乘客行程:** 每个航班的乘客数量和每张机票的价格。这些信息对于评估每个航班带来的收入非常有用。

请注意,尽管本示例中使用的数据来自2006年,优化模型对数据是无关的。
对于任何新的航班计划和预测的未来中断水平,模型将优化解决航线和航班服务决策。
 

### 包
首先,安装并导入处理数据所需的Python包。

In [6]:
# %pip install networkx matplotlib seaborn
import pandas as pd    
import matplotlib.pyplot as plt  
import seaborn as sns  
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import random
import csv
import plotly.express as px

### 无中断的航班计划

接下来,读取中断当天(即2006年1月7日)的计划航班时刻表。此外,我们推断每个航班的起始和目的机场,以及起始时间(航班从起始机场起飞的时间)和结束时间(航班到达目的机场的时间)。
我们将其存储在一个Pandas数据框中。



In [10]:
df_current_plan = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/flight_rotations_2006-07-01.csv') 
# 如果您在本地运行此笔记本,也可以使用
#df_current_plan = pd.read_csv('data/flight_rotations_2006-07-01.csv') 

df_current_plan['start_time'] = pd.to_datetime(df_current_plan['start_time'], format='%H:%M')
df_current_plan['start_time'] = df_current_plan['start_time'].dt.time
df_current_plan['end_time'] = pd.to_datetime(df_current_plan['end_time'], format='%H:%M')
df_current_plan['end_time'] = df_current_plan['end_time'].dt.time
df_current_plan['duration'] = pd.to_datetime(df_current_plan['duration'], format='%H:%M')
df_current_plan['duration'] = df_current_plan['duration'].dt.time
df_current_plan

此数据集中有多少**航班**、**机场**和**飞机**?

In [11]:
flights = df_current_plan['flight'].unique()
aircrafts = df_current_plan['aircraft'].unique()
airports = set(df_current_plan['ori'].unique()+df_current_plan['des'].unique())

print(len(flights),"flights between",len(airports),"airports operated with",len(aircrafts),"aircrafts")

### 可视化航班网络

接下来,我们汇总所有航班的起始和目的机场,并可视化整个计划航班网络。我们使用networkx将机场间信息存储为*有向图*数据结构以实现此可视化。


此外,为了减少输入的大小,我们可以选择仅选择前几个机场(根据通过它们的航班数量)用于笔记本的其余部分。n_airports参数选择要预选的机场数量,默认设置为20个机场。


下面的视觉效果旨在说明航班地图的复杂性;不必花费太多时间分析它。我们稍后将深入研究具体的航线。

In [12]:
from IPython.display import Image, display
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_agraph import to_agraph 
 
arcs = list(df_current_plan[['ori','des']].itertuples(index=False, name=None)) # 存储所有航班的起点-终点对

n_airports = 4 # 指定选择多少个机场
    
G = nx.MultiDiGraph() # 创建一个空的有向图
G.add_edges_from(arcs) # 将起点-终点对作为有向边添加到图中

top_airports = [i for (i,j) in sorted(G.degree, key=lambda x: x[1], reverse=True)[:n_airports]] # 按度数预选顶级机场

G = G.subgraph(top_airports) # 将图缩减为仅包含顶级机场
 
# 将当前计划数据框缩减为仅包含顶级机场    
df_current_plan = df_current_plan[df_current_plan['ori'].isin(top_airports)] 
df_current_plan = df_current_plan[df_current_plan['des'].isin(top_airports)]

# # 可视化网络
# A_graph = to_agraph(G) 
# A_graph.layout('dot')    
# display(A_graph) 
 

flights = df_current_plan['flight'].unique()
aircrafts = df_current_plan['aircraft'].unique()
airports = set(df_current_plan['ori'].unique()+df_current_plan['des'].unique())

print("The reduced data has",len(flights),"flights between",len(airports),"airports operated with",len(aircrafts),"aircrafts")


对于每个航班,我们将存储其起始机场、目的机场、起始时间和结束时间到字典中。

In [ ]:

flight_origin = df_current_plan.set_index('flight')['ori'].to_dict()
flight_dest = df_current_plan.set_index('flight')['des'].to_dict()
flight_start_time = df_current_plan.set_index('flight')['start_time'].to_dict()
flight_end_time = df_current_plan.set_index('flight')['end_time'].to_dict() 


### 飞机一天开始和结束时的位置

在中断日开始时,每架飞机从一个特定的机场开始,并且必须在一天结束时到达一个特定的机场。这是为了确保飞机机队为第二天的航班运营做好准备。

我们现在读取飞机机队的起始位置(称为**源**)和结束位置(称为**汇**)的信息。 

In [16]:
df_starting_positions = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/starting_positions.csv')  
# 如果您在本地运行此笔记本,也可以使用
#df_starting_positions = pd.read_csv('data/starting_positions.csv')  
aircrafts_startpositions_airc = df_starting_positions.set_index('aircraft')['airport'].to_dict()

df_ending_positions = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/ending_positions.csv')
# 如果您在本地运行此笔记本,也可以使用
#df_ending_positions = pd.read_csv('data/ending_positions.csv')
aircrafts_endpositions_airc = df_ending_positions.set_index('aircraft')['airport'].to_dict()


### 乘客行程

接下来,我们读取乘客行程数据。对于每个航班,我们知道有多少乘客预订了机票以及每个座位的费用。我们将这些信息存储到字典中,以便稍后评估取消航班的成本。

In [ ]:
df_iterinaries = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/flight_iterinaries.csv')
#df_iterinaries = pd.read_csv('data/flight_iterinaries.csv')
df_iterinaries['total_cost'] = df_iterinaries['cost']*df_iterinaries['n_pass']
flight_revenue = df_iterinaries.groupby(['flight'])['total_cost'].agg('sum').to_dict() 
flight_n_pass = df_iterinaries.groupby(['flight'])['n_pass'].agg('sum').to_dict() 
df_iterinaries


### 为每架飞机创建航班到航班的转换

最后,使用当前航班计划,我们评估所有可行的航班到航班的转换。这种转换本质上是:每次航班之后,下一个可能的航班是什么?

对于两个航班 $f_1$ 和 $f_2$,航班转换 $f_1$-$f_2$ 是*可行的*,如果 $f_1$ 的到达时间早于 $f_2$ 的起飞时间,并且 $f_1$ 的目的地与 $f_2$ 的起始地相同。

基于这些可行的转换,每架飞机的航线是从源机场到其汇机场的一系列航班到航班的转换。


例如,从数据中,飞机 A380#1(即空客380)一天开始时位于机场 CFE。从这里,它可以选择航班 4296(CFE-ORY) 在早上5:40或航班 4298(CFE-ORY) 在早上10:48。一旦到达 ORY,它将在当天的航线中有多个选择。总的来说,A380#1 有八条航线(见下图)。

为了紧凑地存储所有可行的航班到航班的转换,我们创建了一个**有向无环图**(DAG)。顶点是航班,有向边是可行的转换。使用下面的交互工具可视化每架飞机的DAG。

In [27]:

from ipywidgets import interact, interactive, fixed, interact_manual

aircraft_flights = df_current_plan.groupby(['aircraft']).apply(lambda x: x['flight'].tolist()).to_dict()
flight_arcs_for_each_aircraft = {}
deltaplus_flightarcs = {}
deltaminus_flightarcs = {}
for a in aircraft_flights:
    aircraft_flights[a] += ['source_%s'%a,'sink_%s'%a]
    flight_origin['source_%s'%a] = aircrafts_endpositions_airc[a]
    flight_dest['source_%s'%a] = aircrafts_startpositions_airc[a]
    flight_origin['sink_%s'%a] = aircrafts_endpositions_airc[a]
    flight_dest['sink_%s'%a] = aircrafts_startpositions_airc[a]

    flight_start_time['source_%s'%a] = datetime.strptime('0:0', '%H:%M').time()
    flight_end_time['source_%s'%a] = datetime.strptime('0:0', '%H:%M').time()

    flight_start_time['sink_%s'%a] = datetime.strptime('23:59', '%H:%M').time()
    flight_end_time['sink_%s'%a] = datetime.strptime('23:59', '%H:%M').time()

    flight_arcs_for_each_aircraft[a] = []
    deltaplus_flightarcs[a] = {f: [] for f in aircraft_flights[a]}
    deltaminus_flightarcs[a] = {f: [] for f in aircraft_flights[a]}

    for f1 in aircraft_flights[a]:
        for f2 in aircraft_flights[a]:
            if f1!=f2 and flight_end_time[f1] < flight_start_time[f2] and flight_dest[f1] == flight_origin[f2]: 
                flight_arcs_for_each_aircraft[a].append((f1,f2))
                deltaplus_flightarcs[a][f1].append(f2)
                deltaminus_flightarcs[a][f2].append(f1) 
            # 允许直接连接源和目标的情况,适用于飞机完全不使用的情况    
            elif str(f1).startswith('source') and str(f2).startswith('sink'):
                flight_arcs_for_each_aircraft[a].append((f1,f2))
                deltaplus_flightarcs[a][f1].append(f2)
                deltaminus_flightarcs[a][f2].append(f1)

In [ ]:
# 需要用于可视化DAG的包
# 如有问题请取消注释整个单元格
!apt install libgraphviz-dev
!pip install pygraphviz 

def visualize_aircraft_network(x): 
    G = nx.DiGraph()
    G.add_edges_from(flight_arcs_for_each_aircraft[x])
    plt.figure(figsize=(20,14)) 
    A_graph = to_agraph(G) 
    A_graph.layout('dot')    
    display(A_graph)             
    plt.show()
 
interact(visualize_aircraft_network, x=aircraft_flights.keys())

## 优化模型

 
天气中断减少了机场的整体容量,以航班起飞和降落的数量来衡量。
鉴于这种减少的机场容量,哪些航班应该运营,飞机应该采取什么路线?
我们的目标是创建一个最优航班计划,以最小化因取消航班而产生的整体收入损失。

这个决策问题是使用数学优化模型来建模的,该模型根据**目标函数**找到**最佳解决方案**,使解决方案满足一组**约束条件**。
在这里,解决方案表示为一组实值或整数值,称为**决策变量**。
约束条件是一组以决策变量为函数的方程或不等式。

在这个航空公司业务模型中,目标是最小化因取消航班而产生的整体损失。
决策变量决定哪些航班运营/取消,以及为每架飞机构建一个从其起始机场到一天结束时需要到达的机场的路线。
有三种类型的约束条件:(i)构建航线,(ii)确保航班仅在航线中运营,(iii)确保起飞和降落的数量在机场减少的容量范围内。

### 假设

在本笔记本中做出了许多建模假设,因为此模型是一个起点。在笔记本的末尾,我们提出了潜在的扩展建议。
以下是一些关键假设。
- 所有机场在一天内具有相同的中断水平。
- 我们假设提前知道所有机场的中断水平。
- 我们忽略了机组人员调度和维护问题;尽管可以使用商业Gurobi许可证扩展此模型以处理更大的输入。
- 我们没有考虑其他航空公司可能对中断的反应。



### 输入参数

现在让我们定义用于创建模型的输入参数和符号。下标$a$将用于表示每架飞机,$f$表示每个航班,$i$表示每个机场。


- $N$: 所有机场的集合
- $A$: 所有飞机的集合
- $F$: 所有航班的集合
- $F_a$: 当前计划中由飞机$a$运营的航班集合
- $E_a$: 飞机$a$的可行航班到航班转换集合
- $r_f$: 航班$f$的收入($\$$)
- $(o_f,d_f)$: 航班$f$的起始机场和目的机场
- $(C^{arr}_i,C^{dep}_i)$: 机场$i$的最大到达和起飞数量
- $\alpha$: 中断水平

以下代码加载了Gurobi Python包并启动了优化模型。
$\alpha$的值设置为$50\%$。
 

In [28]:
# %pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("airline_disruption")
 
N = G.nodes() 

### 决策变量

现在我们定义决策变量。
在我们的模型中,我们想做两件事:选择每架飞机运营的航班并构建每架飞机的航线。
以下符号用于建模这些决策变量。


$x_{a,f}$: $1$, 如果飞机$a$运营航班$f$; $0$, 否则

$y_{a,f_1,f_2}$: $1$, 如果飞机$a$在航班$f_1$之后立即运营航班$f_2$; $0$, 否则

我们将使用addVar函数将变量添加到Gurobi模型中。

In [29]:
x, y = {}, {}
for a in aircrafts:
    for f in aircraft_flights[a]:
        x[a,f] = model.addVar(name="x_%s,%s"%(a,f), vtype=GRB.BINARY)

    for (f1,f2) in flight_arcs_for_each_aircraft[a]:
        y[a,f1,f2] = model.addVar(name="y_%s,%s,%s"%(a,f1,f2), vtype=GRB.BINARY)

model.update()


### 设置目标: 最小化因取消航班造成的收入损失

<!-- Next, we will define the objective function: we want to maximizing the **net revenue**. The revenue from sales in each region is calculated by the price of an avocado in that region multiplied by the quantity sold there. There are two types of costs incurred: the wastage costs for excess unsold avocados and the cost of transporting the avocados to the different regions. 

The net revenue is the sales revenue subtracted by the total costs incurred. We assume that the purchase costs are fixed and are not incorporated in this model. -->

我们的目标是**最小化**因取消航班造成的总**收入损失**。
我们将此目标捕获为决策变量的函数。
请注意,如果$x_{a,f}$设置为$0$,则航班被取消。
航班的收入损失由$(1-x_{a,f}) * r_f$给出。
因此,所有飞机和取消航班的整体收入损失由以下公式给出:


<!-- \begin{aligned} 
\textrm{Maximize } \ \sum_{f \in flights} \ \sum_{a \in aircrafts} \ x_{a,f} * r_f
\end{aligned} -->

\begin{aligned} 
\textrm{Minimize } \ \sum_{a \in aircrafts} \ \sum_{f \in F_a} \ (1-x_{a,f}) * r_f
\end{aligned}

我们现在使用setObjective函数将此目标函数添加到模型中。

 

In [ ]:
objective = gp.quicksum((1-x[a,f])*flight_revenue[f] for a in aircrafts for f in aircraft_flights[a] if f in flight_revenue) # 运营成本
model.setObjective(objective, sense=GRB.MINIMIZE)

### 约束条件 #1: 构建每架飞机的航线


飞机从其起始机场(源)开始一天,并在一天结束时到达其最终机场(汇)。其当天的航线是使用**y**决策变量构建的。

我们通过考虑每架飞机的三种情况来实现这一点:其起始航班、中间航班和结束航班。

当航班离开其起始机场时,我们确保它可以离开一次。
集合$\delta^+(source_a)$给出了飞机的所有候选"第一航班"的集合。
我们确保其中一个航班被选中,使用以下等式对每架飞机$a$进行约束。

\begin{aligned} 
\sum_{f' \in \delta^+(source_a)} y_{a,source_a,f'} &= 1
\end{aligned} 

类似地,当航班到达其最终机场时,我们确保它进入机场一次。
集合$\delta^-(sink_a)$给出了飞机的所有候选"最后航班"的集合,我们确保其中一个航班被选中。

\begin{aligned} 
\sum_{f' \in \delta^-(sink_a)} y_{a,f',sink_a} &= 1
\end{aligned}

对于每个中间航班$f$在$F_a$中(既不是起始航班也不是结束航班),我们确保前后航班的数量相同。这对于确保航线的连续性是必要的。
以下约束条件适用于每架飞机$a$和$F_a$中的中间航班$f$。

\begin{aligned} 
\sum_{f' \in \delta^+(f)} y_{a,f,f'} &= \sum_{f' \in \delta^-(i)} y_{a,f',f} 
\end{aligned}

在优化建模中,这些类型的约束条件称为**流量平衡**约束条件。
这些用于建模许多著名问题,如最短路径、最大流量问题和旅行商问题。阅读更多[这里](https://web.mit.edu/15.053/www/AMP-Chapter-08.pdf)。
以下代码将这些约束条件逐一添加到模型中。

In [22]:
for a in aircrafts:
    model.addConstr(sum(y[a,'source_%s'%a,f2] for f2 in deltaplus_flightarcs[a]['source_%s'%a]) == 1)
    model.addConstr(sum(y[a,f1,'sink_%s'%a] for f1 in deltaminus_flightarcs[a]['sink_%s'%a]) == 1)
    for f in aircraft_flights[a]: 
        if str(f)[0] != 's':
            model.addConstr(sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f]) == sum(y[a,f1,f] for f1 in deltaminus_flightarcs[a][f]))

### 约束条件 #2: 航班仅在被飞机穿越时运营

接下来,我们确保航班$f$仅在飞机$a$的航线中时由飞机$a$运营。
数量$\sum_{f' \in \delta^+(f)} y_{a,f,f'}$给出了离开航班$f$的弧的数量;可以是$0$弧或$1$弧。
如果此数量为$0$,则飞机$f$不穿越航班$f$,我们将$x_{a,f}$设置为$0$。
此约束条件可以通过以下不等式对每架飞机$a$和航班$f$在$F_a$中进行数学表达。

\begin{aligned}  
x_{a,f} &\leq \sum_{f'\ \textrm{in }\ \delta^+(f)} y_{a,f,f'}
\end{aligned}

现在让我们将这些约束条件添加到模型中。


In [23]:
for a in aircrafts:
    for f in aircraft_flights[a]:
        model.addConstr(x[a,f] <= sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f])) # flight f is chosen only if it is traversed
        

### 约束条件 #3: 机场的最大到达和起飞数量限制

最后,我们添加机场容量约束条件。
对于每个机场,我们知道正常情况下的总到达和起飞数量。
然而,在中断日,只有一部分航班可以降落和起飞,由参数$\alpha$给出。
例如,如果$\alpha = 0.5$,则只有一半的航班可以降落或起飞。
此条件可以通过以下不等式对每个机场$i$进行数学表达。

\begin{aligned} 
\sum_{\textrm{aircraft a}} \ \sum_{\textrm{flight }f \textrm{ that arrives at $i$}} x_{a,f} &\leq C^{arr}_{i} * \alpha \quad  \forall \ \textrm{airport } i, \\
\sum_{\textrm{aircraft a}} \ \sum_{\textrm{flight }f \textrm{ that departs from $i$}} x_{a,f} &\leq C^{dep}_{i} *\alpha \quad  \forall \ \textrm{airport } i.
\end{aligned}

不等式的左侧计算机场降落或起飞的航班总数,右侧设置最大限制。
作为极端情况,设置$\alpha  = 0$意味着机场完全关闭,设置$\alpha = 1$意味着没有中断。

我们可以将这些约束条件添加到模型中,默认值设置为$0.5$。稍后在笔记本中,我们将看到中断参数如何影响最优航班计划。


In [24]:
alpha = .5 

for i in N:
    total_departures = len([f for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i])
    total_arrivals = len([f for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i])

    model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i) <= alpha*total_departures)
    model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i) <= alpha*total_arrivals)


### 启动Gurobi引擎

我们已经将决策变量、目标函数和约束条件添加到模型中。
模型已准备好求解。 

In [25]:
model.optimize()

### 最优解

求解器在不到一秒的时间内解决了优化问题。
现在让我们分析最优解。

In [ ]:
operated_flights = {a: [f for f in aircraft_flights[a] if x[a,f].X > .5 if str(f)[0] != 's'] for a in aircrafts}
actual_rev = sum(flight_revenue[f] for f in flight_revenue) # 运营成本

print("\nNet revenue total loss: $",round(model.objVal/10**6,2),'million')
print("Optimal number of flights served:",sum(len(operated_flights[a]) for a in aircrafts))
print("Optimal number of passengers transported:",sum(sum(flight_n_pass[f] for f in aircraft_flights[a] if x[a,f].X > .5) for a in aircrafts))
print("Optimal number of aircrafts utilized:",sum([1 if len(operated_flights[a]) > 0 else 0 for a in aircrafts]))


## 完整模型

虽然本笔记本逐步介绍了如何构建优化模型,但以下代码包含了整体优化模型。您可以输入不同的参数值,看看最优解如何变化。
$\alpha$(百分比中断)的值可以通过单元格下方的滑块进行控制。


In [ ]:
import gurobipy as gp
from gurobipy import GRB
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
 
N = G.nodes() 
 
def solve_flight_planning(x):    
    alpha = x 
    
    model = gp.Model("airline_disruption")
    x, y = {}, {}
    for a in aircrafts:
        for f in aircraft_flights[a]:
            x[a,f] = model.addVar(name="x_%s,%s"%(a,f), vtype=GRB.BINARY)

        for (f1,f2) in flight_arcs_for_each_aircraft[a]:
            y[a,f1,f2] = model.addVar(name="y_%s,%s,%s"%(a,f1,f2), vtype=GRB.BINARY)

    model.update()

    objective = gp.quicksum((1-x[a,f])*flight_revenue[f] for a in aircrafts for f in aircraft_flights[a] if f in flight_revenue) # 运营成本
    model.setObjective(objective, sense=GRB.MINIMIZE)

    for a in aircrafts:
        model.addConstr(sum(y[a,'source_%s'%a,f2] for f2 in deltaplus_flightarcs[a]['source_%s'%a]) == 1)
        model.addConstr(sum(y[a,f1,'sink_%s'%a] for f1 in deltaminus_flightarcs[a]['sink_%s'%a]) == 1)
        for f in aircraft_flights[a]: 
            if str(f)[0] != 's':
                model.addConstr(sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f]) == sum(y[a,f1,f] for f1 in deltaminus_flightarcs[a][f]))

    for a in aircrafts:
        for f in aircraft_flights[a]:
            model.addConstr(x[a,f] <= sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f])) # flight f is chosen only if it is traversed


    for i in N:
        total_departures = len([f for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i])
        total_arrivals = len([f for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i]) 

        model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i) <= alpha*total_departures)
        model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i) <= alpha*total_arrivals)

    model.setParam('OutputFlag', 0)
    model.optimize()

    operated_flights = {a: [f for f in aircraft_flights[a] if x[a,f].X > .5 if str(f)[0] != 's'] for a in aircrafts}
    actual_rev = sum(flight_revenue[f] for f in flight_revenue) # 运营成本
    print("Loss ($): ",round(model.objVal/10**6,2),'million')
    print("Optimal number of flights served:",sum(len(operated_flights[a]) for a in aircrafts))
    print("Optimal number of passengers transported:",sum(sum(flight_n_pass[f] for f in aircraft_flights[a] if x[a,f].X > .5) for a in aircrafts))
    print("Optimal number of aircrafts utilized:",sum([1 if len(operated_flights[a]) > 0 else 0 for a in aircrafts]))
    
    # 以下几行用于可视化,需要pygraphviz包(如果有问题可以注释掉)
    print("已运营航班的完整网络:")
    G = nx.MultiDiGraph()
    arcs=[(flight_origin[f],flight_dest[f]) for a in aircrafts for f in operated_flights[a]] 
     
    aircraft_color = {aircrafts[i]:"#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(aircrafts))}
    for a in aircrafts:
        for f in operated_flights[a]:
            G.add_edge(flight_origin[f],flight_dest[f],color=aircraft_color[a])
    A_graph = to_agraph(G) 
    A_graph.layout('dot')  
    display(A_graph)

print("Select a value for the level of disruption at the airports:\n")
print("Select 0 for complete shutdown of all airports; select 1 for business-as-usual.\n")

interact(solve_flight_planning, x=(0,1,0.05)) 
      

In [32]:
model.dispose()
gp.disposeDefaultEnv()

## 扩展

- 除了取消航班,还有其他方法可以在中断后更改时刻表,例如延迟航班、将乘客重新预订到其他航班等。您可以使用决策变量对这些进行建模吗?需要什么类型的约束条件?
- 我们只考虑了取消航班的成本。可能还有其他成本,如维护和修理成本,我们没有考虑。如何将其他成本纳入模型?
- 此模型未包括与机组人员计划相关的间接考虑。例如,当预备机组人员由于航班取消而错过连接时,该机组人员将无法服务未来的航班。机组人员计划也可以在模型中捕获吗?


Copyright © 2023 Gurobi Optimization, LLC